In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, LSTM, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import optuna


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, LSTM, MultiHeadAttention
from tensorflow.keras.layers import Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import optuna

class FeatureSelector:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

class MultiModalFeatureExtractor:
    def __init__(self):
        self.mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        self.resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        self.efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        # Freeze base models
        for model in [self.mobilenet, self.resnet, self.efficientnet]:
            for layer in model.layers:
                layer.trainable = False

def create_combined_model(trial):
    # Hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    dense_units = trial.suggest_int('dense_units', 64, 512)
    
    # Input
    input_layer = Input(shape=(224, 224, 3))
    
    # Feature extractors
    mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze base models
    for model in [mobilenet, resnet, efficientnet]:
        for layer in model.layers:
            layer.trainable = False
    
    # Extract features
    mobilenet_features = mobilenet(input_layer)
    resnet_features = resnet(input_layer)
    efficientnet_features = efficientnet(input_layer)
    
    # Global pooling
    mobilenet_pooled = GlobalAveragePooling2D()(mobilenet_features)
    resnet_pooled = GlobalAveragePooling2D()(resnet_features)
    efficientnet_pooled = GlobalAveragePooling2D()(efficientnet_features)
    
    # Concatenate features
    concatenated = Concatenate()([mobilenet_pooled, resnet_pooled, efficientnet_pooled])
    
    # Dense layers
    x = Dense(dense_units, activation='relu')(concatenated)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    x = Dense(dense_units // 2, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Output layers
    jenis_output = Dense(2, activation='softmax', name='jenis')(x)
    warna_output = Dense(5, activation='softmax', name='warna')(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=[jenis_output, warna_output])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss={
            'jenis': 'sparse_categorical_crossentropy',
            'warna': 'sparse_categorical_crossentropy'
        },
        metrics={
            'jenis': 'accuracy',
            'warna': 'accuracy'
        }
    )
    
    return model

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        img_path = image_path + '.jpg'
        img = Image.open(img_path)
    except:
        img_path = image_path + '.png'
        img = Image.open(img_path)
    
    img = img.convert('RGB')
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

def create_dataset(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    images = []
    
    for idx in df['id']:
        img_path = os.path.join(img_dir, str(idx))
        img_array = load_and_preprocess_image(img_path)
        images.append(img_array)
    
    return np.array(images), df

def objective(trial):
    # Load and prepare data
    X_train, train_df = create_dataset(
        './datas/train.csv',
        './datas/train/train'
    )
    
    # Split data
    X_train, X_val, y_jenis_train, y_jenis_val, y_warna_train, y_warna_val = train_test_split(
        X_train,
        train_df['jenis'].values,
        train_df['warna'].values,
        test_size=0.2,
        random_state=42
    )


    trial = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

# Create and train model
    model = create_combined_model(trial)
    
    # Training callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        )
    ]
    
    # Train
    history = model.fit(
        X_train,
        {'jenis': y_jenis_train, 'warna': y_warna_train},
        validation_data=(
            X_val,
            {'jenis': y_jenis_val, 'warna': y_warna_val}
        ),
        epochs=16,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )
    
    # Return combined validation accuracy
    val_jenis_acc = max(history.history['val_jenis_accuracy'])
    val_warna_acc = max(history.history['val_warna_accuracy'])
    return (val_jenis_acc + val_warna_acc) / 2

def main():
    # GPU configuration
    try:
        physical_devices = tf.config.list_physical_devices('GPU')
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
    except:
        print("No GPU devices found. Using CPU.")
    
    # Optuna study
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=4)
    
    # Print best parameters
    print("Best hyperparameters:", study.best_params)
    
    # Train final model with best parameters
    print("Training final model...")
    
    # Load all data
    X_train, train_df = create_dataset(
        './datas/train.csv',
        './datas/train/train'
    )
    
    # Create and train final model
    final_model = create_combined_model(study.best_trial)
    
    # Final training
    final_model.fit(
        X_train,
        {'jenis': train_df['jenis'].values, 'warna': train_df['warna'].values},
        epochs=50,
        batch_size=32,
        callbacks=[
            EarlyStopping(patience=15, restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=7, min_lr=1e-6)
        ],
        verbose=1
    )
    
    # Predict test data
    X_test, test_df = create_dataset(
        './datas/sample_submission.csv',
        './datas/test/test'
    )
    
    jenis_pred, warna_pred = final_model.predict(X_test)
    
    # Create submission
    submission = pd.DataFrame({
        'id': test_df['id'],
        'jenis': np.argmax(jenis_pred, axis=1),
        'warna': np.argmax(warna_pred, axis=1)
    })
    
    submission.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")

if __name__ == "__main__":
    main()

[I 2024-10-25 17:22:08,288] A new study created in memory with name: no-name-0b1d4081-d443-48b4-ab4a-ff1938d33c99
[W 2024-10-25 17:22:10,512] Trial 0 failed with parameters: {} because of the following error: AttributeError("'ImageDataGenerator' object has no attribute 'suggest_float'").
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Hology/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/9d/fp3wp_nx6xqbblmg0zr1yk700000gn/T/ipykernel_27433/2897322130.py", line 164, in objective
    model = create_combined_model(trial)
  File "/var/folders/9d/fp3wp_nx6xqbblmg0zr1yk700000gn/T/ipykernel_27433/2897322130.py", line 50, in create_combined_model
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
AttributeError: 'ImageDataGenerator' object has no attribute 'suggest_float'
[W 2024-10-25 17:22:10,515] Trial 0 failed with value None.


AttributeError: 'ImageDataGenerator' object has no attribute 'suggest_float'